# cGAN Generate Synthetic Data for German Dataset

CTGAN model is based on the GAN-based Deep Learning data synthesizer 

In [1]:
from implementation_functions import *

import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose

import pandas as pd
import numpy as np
from prince import FAMD #Factor analysis of mixed data
from aif360.metrics import BinaryLabelDatasetMetric
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import skfuzzy as fuzz

In [2]:
data_name = "german"
dataset_orig, privileged_groups, unprivileged_groups = aif_data(data_name, False)

In [3]:
sens_attr = ['age', 'sex']
decision_label = 'credit'
fav_l = 1
unfav_l = 0

In [4]:
orig_df, num_list, cat_list = preprocess(dataset_orig, sens_attr, decision_label)

# The list of sub-group sizes in the dataset (to monitor the dist. of sub-groups)
orig_df['sub_labels'].value_counts()

7    447
6    158
3    143
2     62
1     58
5     52
0     47
4     33
Name: sub_labels, dtype: int64

In [5]:
import time
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

--- 3.147125244140625e-05 seconds ---


In [6]:
# Train-test split WITH stratification
X = orig_df.loc[:, orig_df.columns != decision_label]
y = orig_df.loc[:, orig_df.columns == decision_label].values
X_train, X_test, y_train, y_txest = train_test_split(X, y, test_size=0.30, 
                                                    shuffle=True,
                                                    stratify=X['sub_labels'])

In [15]:
X_train_new = X_train.drop(['age', 'sex', 'sub_labels'], axis=1)

In [14]:
#X_train_new['class_labels'] = y_train

# Here we start the GAN work

In [ ]:
from sdv.tabular import CTGAN
model = CTGAN()

In [ ]:
from sdv.tabular import CTGAN
model = CTGAN()
start_time = time.time()
model.fit(X_train)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
loaded = CTGAN.load('my_fariness_German_V1.pkl')

Generating X_train, the following cell shows 

In [ ]:
available_rows = {}
for row_count in range(8):
    available_rows[row_count] = X_train["sub_labels"].value_counts()[row_count]
        
target_rows = max(available_rows.values())
max_label = max(available_rows, key=available_rows.get)
print(target_rows)
print(max_label)

In [ ]:
main_df = pd.DataFrame()

In [ ]:
for key, value in available_rows.items():
    if int(key) != int(max_label):
        conditions = {
            "sub_labels" : int(key),
        }
        needed_rows = target_rows - value
        main_df = pd.concat([main_df, loaded.sample(needed_rows, conditions=conditions)])
        
print(len(main_df.index))

In [ ]:
main_df

In [ ]:
print(type(main_df))

In [ ]:
main_df.to_csv('german_synth.csv', index = False, header=True)

not to use any of the sensetive attributes ...trying to be blind by not using any of these obvious attributes ... we delete the sent att .. but still there is bias. Deleting them, will not remove the bias, but as a first step we are tryring to delete the traces of bias by deleteing sent and the techq

In [ ]:
# Keep the subgroup labels to append them back later
keep_sub_l = X_train['sub_labels']

# Required drops for the GERMAN dataset (THIS DF CREATION IS A MUST)
X_train_new = X_train.drop(['age', 'sex', 'sub_labels'], axis=1)

# Get the idx of categ and numeric columns again due to the column drops above
num_list, cat_list = type_lists(X_train_new)

In [ ]:
# Type the desired classifier to train the classification models with model obj
clf = GradientBoostingClassifier()
baseline_stats, cm, ratio_table, preds = baseline_metrics(clf, X_train, X_test, 
                                                  y_train, y_test, sens_attr, 
                                                  fav_l, unfav_l)

In [ ]:
X_train_new

In [ ]:
clf = RandomForestClassifier()   
baseline_stats, cm, ratio_table, preds = baseline_metrics(clf, X_train, X_test, 
                                                  y_train, y_test, sens_attr, 
                                                  fav_l, unfav_l)

-str2: clusters membership-->each sample for cluster.. 
-str3: all of the clusters, each trained clusters * pp

In [ ]:
print(type(main_df))

In [ ]:
print(type(X_test))

In [ ]:
print(type(clf))

In [ ]:
print(ratio_table)

In [ ]:
test_sublabels = X_test['sub_labels']
X_test_n = X_test.drop(['age', 'sex','sub_labels'], axis=1)
num_list, cat_list = type_lists(X_test_n)

In [ ]:
# Predicting the test sets based on strategy 1
X_test_pred1 = predict_whole_set(clf, main_df, X_test_n)

print(dict_german["month"])

In [ ]:
metrics_table1, cm1, ratio_t1 = metrics_calculate(X_test, X_test_pred1, y_test,
                                                  sens_attr, fav_l, unfav_l)
metrics_table2, cm2, ratio_t2 = metrics_calculate(X_test, X_test_pred2, y_test,
                                                  sens_attr, fav_l, unfav_l)
metrics_table3, cm3, ratio_t3 = metrics_calculate(X_test, X_test_pred3, y_test,
                                                  sens_attr, fav_l, unfav_l)

In [ ]:
#outputs from strategy 1
print(metrics_table1)
print("Confusion Matrix:", cm1)
print(ratio_t1)

In [ ]:
#outputs from strategy 2
print(metrics_table2)
print("Confusion Matrix:", cm2)
print(ratio_t2)

In [ ]:
#outputs from strategy 3
print(metrics_table3)
print("Confusion Matrix:", cm3)
print(ratio_t3)